In [1]:
import pandas as pd
import numpy as np

In [22]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/refs/heads/master/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [7]:
X = df.iloc[:,: -1].values
y = df.iloc[:, -1].values


In [8]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
X = scaler.fit_transform(X)

In [11]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

In [87]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [18]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7722 - loss: 0.4914 - val_accuracy: 0.7987 - val_loss: 0.4734
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7411 - loss: 0.4815 - val_accuracy: 0.7987 - val_loss: 0.4717
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7476 - loss: 0.5145 - val_accuracy: 0.7922 - val_loss: 0.4695
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7775 - loss: 0.4688 - val_accuracy: 0.7987 - val_loss: 0.4674
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7947 - loss: 0.4667 - val_accuracy: 0.7922 - val_loss: 0.4670
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7745 - loss: 0.4613 - val_accuracy: 0.7922 - val_loss: 0.4658
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7698 - loss: 0.4718 - val_accuracy: 0.7922 - val_loss: 0.4637
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7763 - loss: 0.4650 - val_accuracy: 0.7922 - 

In [24]:
# 1. How to select appropriate optimiser
import keras_tuner as kt

In [29]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [36]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5
)

Reloading Tuner from ./untitled_project/tuner0.json


In [37]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [38]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

In [40]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7205 - loss: 0.5357 - val_accuracy: 0.7727 - val_loss: 0.5174
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7308 - loss: 0.5178 - val_accuracy: 0.7727 - val_loss: 0.5029
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7470 - loss: 0.4936 - val_accuracy: 0.7727 - val_loss: 0.4927
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7679 - loss: 0.4777 - val_accuracy: 0.7727 - val_loss: 0.4850
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7538 - loss: 0.4952 - val_accuracy: 0.7727 - val_loss: 0.4787
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7817 - loss: 0.4869 - val_accuracy: 0.7792 - val_loss: 0.4748
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7873 - loss: 0.4606 - val_accuracy: 0.7792 - val_loss: 0.4710
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7743 - loss: 0.4631 - val_accuracy: 0.77

In [64]:
# How to select number of nodes in a layer

def build_model(hp):
  model = Sequential()

  units=hp.Choice('units', [8, 16, 32, 48, 64, 120])
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [65]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite=True
)

In [66]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 17s


In [67]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [68]:
model = tuner.get_best_models(num_models=1)[0]

In [70]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7804 - loss: 0.4611 - val_accuracy: 0.8182 - val_loss: 0.4735
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7561 - loss: 0.4746 - val_accuracy: 0.8052 - val_loss: 0.4717
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7513 - loss: 0.4776 - val_accuracy: 0.8052 - val_loss: 0.4708
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8011 - loss: 0.4166 - val_accuracy: 0.7922 - val_loss: 0.4683
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7937 - loss: 0.4343 - val_accuracy: 0.7987 - val_loss: 0.4660
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7589 - loss: 0.4827 - val_accuracy: 0.7987 - val_loss: 0.4662
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7309 - loss: 0.4735 - val_accuracy: 0.8052 - val_loss: 0.4652
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7977 - loss: 0.4213 - val_accuracy: 0.8

In [71]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,404 (9.39 KB)

 Trainable params: 1,201 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,203 (4.70 KB)

In [72]:
# 3. How to select number of layers

def build_model(hp):
  model = Sequential()
  model.add(Dense(72, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    model.add(Dense(72, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [73]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite=True
)

In [74]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 26s


In [75]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [76]:
model = tuner.get_best_models(num_models=1)[0]

In [77]:
model.fit(X_train, y_train, epochs=100, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.7686 - loss: 0.4820 - val_accuracy: 0.7078 - val_loss: 0.5673
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7811 - loss: 0.4382 - val_accuracy: 0.7532 - val_loss: 0.5131
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7824 - loss: 0.4613 - val_accuracy: 0.8312 - val_loss: 0.4571
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7613 - loss: 0.4562 - val_accuracy: 0.7922 - val_loss: 0.5007
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8070 - loss: 0.4121 - val_accuracy: 0.7597 - val_loss: 0.6365
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7803 - loss: 0.4320 - val_accuracy: 0.8247 - val_loss: 0.4717
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7906 - loss: 0.4307 - val_accuracy: 0.7727 - val_loss: 0.5673
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8504 - loss: 0.3570 - val_accuracy: 0.81

In [88]:
# 4. Tuning all Hyperparameters

def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:

      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
          )
      )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:

      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
          )
      )

    counter += 1

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [89]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite=True
)

In [90]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 26s


In [91]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 48,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 32,
 'activation1': 'sigmoid',
 'units2': 40,
 'activation2': 'tanh',
 'units3': 64,
 'activation3': 'sigmoid',
 'units4': 72,
 'activation4': 'relu',
 'units5': 40,
 'activation5': 'sigmoid',
 'units6': 8,
 'activation6': 'sigmoid',
 'units7': 8,
 'activation7': 'tanh',
 'units8': 40,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'relu'}

In [92]:
model = tuner.get_best_models(num_models=1)[0]

In [93]:
model.fit(X_train, y_train, epochs=100, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7511 - loss: 0.5183 - val_accuracy: 0.7597 - val_loss: 0.4870
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7604 - loss: 0.4955 - val_accuracy: 0.7792 - val_loss: 0.4772
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7452 - loss: 0.4882 - val_accuracy: 0.7987 - val_loss: 0.4695
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7636 - loss: 0.4727 - val_accuracy: 0.7857 - val_loss: 0.4664
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7830 - loss: 0.4588 - val_accuracy: 0.7727 - val_loss: 0.4962
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7816 - loss: 0.4796 - val_accuracy: 0.7792 - val_loss: 0.4723
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7631 - loss: 0.4693 - val_accuracy: 0.7922 - val_loss: 0.4711
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7495 - loss: 0.4935 - val_accuracy: 0.80